In [2]:
#authentication
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace

ia = InteractiveLoginAuthentication(tenant_id='16b3c013-d300-468d-ac64-7eda0820b6d3')

# You can find tenant id under azure active directory->properties
ws = Workspace.get(name='Prod',
                     subscription_id='fe38c376-b42a-4741-9e7c-f5d7c31e5873',
                     resource_group='ProdRG',auth=ia)

## ** Create Tabular Dataset from WEB URL (CSV files) and Use Unregistered Dataset Directly in Training Script**

In [6]:
#creates an unregistered TabularDataset from a WEB URL.

from azureml.core.dataset import Dataset

web_path ='https://dprepdata.blob.core.windows.net/demo/Titanic.csv'
titanic_ds = Dataset.Tabular.from_delimited_files(path=web_path)
titanic_ds.take(3).to_pandas_dataframe()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S


In [7]:
#Set data schema
#when you create a TabularDataset, column data types are inferred automatically. If the inferred types don't match your expectations, you can update your dataset schema by specifying column types 

from azureml.data.dataset_factory import DataType

titanic_ds = Dataset.Tabular.from_delimited_files(path=web_path, set_column_types={'Survived': DataType.to_bool()})

# preview the first 3 rows of titanic_ds
titanic_ds.take(3).to_pandas_dataframe()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S


In [ ]:
'''import os
# Create a folder for the script files
script_folder = 'workshop_examples'
os.makedirs(script_folder, exist_ok=True)

print(script_folder)'''

In [ ]:
'''
%%writefile $script_folder/train_titanic.py
#access data in training script

import argparse
from azureml.core import Dataset, Run

parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str)
args = parser.parse_args()

run = Run.get_context()
ws = run.experiment.workspace

# get the input dataset by ID
dataset = Dataset.get_by_id(ws, id=args.input_data)

# load the TabularDataset to pandas DataFrame
df = dataset.to_pandas_dataframe()
'''

In [ ]:
'''
# define compute target, add the code
# define environment as myenv., add the code
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder,
                      script='train_titanic.py',
                      # pass dataset as an input with friendly name 'titanic'
                      arguments=['--input-data', titanic_ds.as_named_input('titanic')],
                      compute_target=compute_target,
                      environment=myenv)
                             
# Submit the run configuration for your training run
run = experiment.submit(src)
run.wait_for_completion(show_output=True)
'''

## Create Tabular Dataset from Datastore (PARQUET files) and Register

In [3]:
#creates an unregistered TabularDataset from a DATASTORE  + PARQUET 

from azureml.core import Workspace, Datastore, Dataset

datastore_name = ws.get_default_datastore()
weather_ds = 'weather-data-florida'

#load data to blob storage from local folder (weather-data) to be able to create a dataset
datastore_name.upload('weather_data', weather_ds, overwrite=True, show_progress=True)

datastore_path = [(datastore_name, weather_ds + '/*/*/data.parquet')]
dataset        = Dataset.Tabular.from_parquet_files(path=datastore_path, partition_format = weather_ds + '/{partition_time:yyyy/MM}/data.parquet')
# **** To read files in .csv or .tsv format, use  from_delimited_files() method. *****
#dataset= Dataset.Tabular.from_delimited_files(path=datastore_path)


"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 10 files
Uploading weather_data/2019/01/data.parquet
Uploaded weather_data/2019/01/data.parquet, 1 files out of an estimated total of 10
Uploading weather_data/2019/02/data.parquet
Uploaded weather_data/2019/02/data.parquet, 2 files out of an estimated total of 10
Uploading weather_data/2019/03/data.parquet
Uploaded weather_data/2019/03/data.parquet, 3 files out of an estimated total of 10
Uploading weather_data/2019/04/data.parquet
Uploaded weather_data/2019/04/data.parquet, 4 files out of an estimated total of 10
Uploading weather_data/2019/05/data.parquet
Uploaded weather_data/2019/05/data.parquet, 5 files out of an estimated total of 10
Uploading weather_data/2019/06/data.parquet
Uploaded weather_data/2019/06/data.parquet, 6 files out of an estimated total of 10
Uploading weather_data/2019/07/data.parquet
Uploaded weather_data/2019/07/data.parquet, 7 files out of an estimated total of 10
Uploading weather_data/2019/08/data.parquet
Uploaded weather_data/201

In [15]:
dataset.to_pandas_dataframe().head()

,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,...,precipDepth,snowDepth,stationName,countryOrRegion,p_k,year,day,version,__index_level_0__,partition_time
0,720735,73805,2019-01-01 00:00:00,30.349,-85.788,21.0,140.0,5.1,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390756,2019-01-01
1,720735,73805,2019-01-01 00:39:00,30.349,-85.788,21.0,150.0,5.7,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390757,2019-01-01
2,720735,73805,2019-01-01 00:53:00,30.349,-85.788,21.0,150.0,4.6,21.1,1019.5,...,0.0,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390758,2019-01-01
3,720735,73805,2019-01-01 01:01:00,30.349,-85.788,21.0,150.0,4.6,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390759,2019-01-01
4,720735,73805,2019-01-01 01:53:00,30.349,-85.788,21.0,140.0,4.1,21.1,1019.6,...,0.0,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390760,2019-01-01


### Wrangle Data for tabular datasets
* keep_columns() and drop_columns() methods.
* filter()


In [21]:
from datetime import datetime
tsd=dataset.filter(dataset['datetime']<datetime(2019,1,2))
#tsd = tsd.time_after(datetime(2019, 1, 1)).time_before(datetime(2019, 1, 10))
tsd.to_pandas_dataframe().head(5)

Method filter: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,...,precipDepth,snowDepth,stationName,countryOrRegion,p_k,year,day,version,__index_level_0__,partition_time
0,720735,73805,2019-01-01 00:00:00,30.349,-85.788,21.0,140.0,5.1,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390756,2019-01-01
1,720735,73805,2019-01-01 00:39:00,30.349,-85.788,21.0,150.0,5.7,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390757,2019-01-01
2,720735,73805,2019-01-01 00:53:00,30.349,-85.788,21.0,150.0,4.6,21.1,1019.5,...,0.0,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390758,2019-01-01
3,720735,73805,2019-01-01 01:01:00,30.349,-85.788,21.0,150.0,4.6,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390759,2019-01-01
4,720735,73805,2019-01-01 01:53:00,30.349,-85.788,21.0,140.0,4.1,21.1,1019.6,...,0.0,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390760,2019-01-01


In [26]:
tsd2 = tsd.keep_columns(columns=['snowDepth', 'datetime', 'partition_time'], validate=False)
tsd2.to_pandas_dataframe().tail()

,datetime,snowDepth,partition_time
154,2019-01-01 20:00:00,NaN,2019-01-01
155,2019-01-01 21:00:00,NaN,2019-01-01
156,2019-01-01 21:00:00,NaN,2019-01-01
157,2019-01-01 22:00:00,NaN,2019-01-01
158,2019-01-01 23:00:00,NaN,2019-01-01


In [24]:
tsd2 = tsd.drop_columns(columns=['wban', 'snowDepth'])
tsd2.take(5).to_pandas_dataframe()

,usaf,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,cloudCoverage,...,precipTime,precipDepth,stationName,countryOrRegion,p_k,year,day,version,__index_level_0__,partition_time
0,720735,2019-01-01 00:00:00,30.349,-85.788,21.0,140.0,5.1,21.1,NaN,None,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390756,2019-01-01
1,720735,2019-01-01 00:39:00,30.349,-85.788,21.0,150.0,5.7,21.1,NaN,None,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390757,2019-01-01
2,720735,2019-01-01 00:53:00,30.349,-85.788,21.0,150.0,4.6,21.1,1019.5,None,...,1.0,0.0,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390758,2019-01-01
3,720735,2019-01-01 01:01:00,30.349,-85.788,21.0,150.0,4.6,21.1,NaN,None,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390759,2019-01-01
4,720735,2019-01-01 01:53:00,30.349,-85.788,21.0,140.0,4.1,21.1,1019.6,None,...,1.0,0.0,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390760,2019-01-01


### Register Tabular Dataset

You can register a new dataset under the same name by creating a new version. A dataset version is a way to bookmark the state of your data so that you can apply a specific version of the dataset for experimentation or future reproductio

In [16]:
#Assign "datetime" column as timestamp and "partition_time" from folder path as partition_timestamp for Tabular Dataset to activate Time Series related APIs. The column to be assigned should be a Date type.
tsd3 = dataset.with_timestamp_columns(timestamp='datetime', partition_timestamp='partition_time')

# register dataset to Workspace
registered_ds = tsd3.register(workspace=ws, 
                            name=weather_ds, 
                            create_new_version=True, 
                            description='Data for Tabular Dataset- time-series.', 
                            tags={ 'type': 'TabularDataset' })

In [8]:
registered_ds.to_pandas_dataframe().head()

,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,...,precipDepth,snowDepth,stationName,countryOrRegion,p_k,year,day,version,__index_level_0__,partition_time
2019-01-01 00:00:00,720735,73805,2019-01-01 00:00:00,30.349,-85.788,21.0,140.0,5.1,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390756,2019-01-01
2019-01-01 00:39:00,720735,73805,2019-01-01 00:39:00,30.349,-85.788,21.0,150.0,5.7,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390757,2019-01-01
2019-01-01 00:53:00,720735,73805,2019-01-01 00:53:00,30.349,-85.788,21.0,150.0,4.6,21.1,1019.5,...,0.0,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390758,2019-01-01
2019-01-01 01:01:00,720735,73805,2019-01-01 01:01:00,30.349,-85.788,21.0,150.0,4.6,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390759,2019-01-01
2019-01-01 01:53:00,720735,73805,2019-01-01 01:53:00,30.349,-85.788,21.0,140.0,4.1,21.1,1019.6,...,0.0,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390760,2019-01-01


### Reload the Dataset from Workspace

In [9]:
# get dataset by dataset name
tsd = Dataset.get_by_name(ws, name=weather_ds)
tsd.to_pandas_dataframe().head(5)
#returns the latest version of the dataset

,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,...,precipDepth,snowDepth,stationName,countryOrRegion,p_k,year,day,version,__index_level_0__,partition_time
2019-01-01 00:00:00,720735,73805,2019-01-01 00:00:00,30.349,-85.788,21.0,140.0,5.1,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390756,2019-01-01
2019-01-01 00:39:00,720735,73805,2019-01-01 00:39:00,30.349,-85.788,21.0,150.0,5.7,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390757,2019-01-01
2019-01-01 00:53:00,720735,73805,2019-01-01 00:53:00,30.349,-85.788,21.0,150.0,4.6,21.1,1019.5,...,0.0,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390758,2019-01-01
2019-01-01 01:01:00,720735,73805,2019-01-01 01:01:00,30.349,-85.788,21.0,150.0,4.6,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390759,2019-01-01
2019-01-01 01:53:00,720735,73805,2019-01-01 01:53:00,30.349,-85.788,21.0,140.0,4.1,21.1,1019.6,...,0.0,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390760,2019-01-01


### Create a dataset from pandas dataframe

In [ ]:
from azureml.core import Workspace, Datastore, Dataset
import pandas as pd

pandas_df = pd.read_csv('<path to your csv file>')
ws = Workspace.from_config()
datastore = Datastore.get(ws, '<name of your datastore>')
dataset = Dataset.Tabular.register_pandas_dataframe(pandas_df, datastore, "dataset_from_pandas_df", show_progress=True)

Method filter: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,...,precipDepth,snowDepth,stationName,countryOrRegion,p_k,year,day,version,__index_level_0__,partition_time
2019-01-01 00:00:00,720735,73805,2019-01-01 00:00:00,30.349,-85.788,21.0,140.0,5.1,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390756,2019-01-01
2019-01-01 00:39:00,720735,73805,2019-01-01 00:39:00,30.349,-85.788,21.0,150.0,5.7,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390757,2019-01-01
2019-01-01 00:53:00,720735,73805,2019-01-01 00:53:00,30.349,-85.788,21.0,150.0,4.6,21.1,1019.5,...,0.0,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390758,2019-01-01
2019-01-01 01:01:00,720735,73805,2019-01-01 01:01:00,30.349,-85.788,21.0,150.0,4.6,21.1,NaN,...,NaN,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390759,2019-01-01
2019-01-01 01:53:00,720735,73805,2019-01-01 01:53:00,30.349,-85.788,21.0,140.0,4.1,21.1,1019.6,...,0.0,NaN,NORTHWEST FLORIDA BEACHES INTL ARPT,US,720735-73805,2019,1,1.0,2390760,2019-01-01


## **Create File Dataset **

Use from_files() method to load files in any format and create un registered filedataset.

In [27]:
#Represents a collection of file references in datastores or public URLs to use in Azure Machine Learning.

from azureml.core.dataset import Dataset

web_paths = [
            'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]

mnist_ds = Dataset.File.from_files(path = web_paths)
mnist_ds.to_path() # list the files

['/http%3A/%2Fyann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz']

In [28]:

mnist_file_dataset = mnist_ds.register(workspace=ws,
                                        name='mnist_opendataset',
                                        description='mnist training and test dataset',
                                        create_new_version=True)

In [18]:
#Create dataset from datastore

{
  "source": [
    "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
    "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
    "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
    "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"
  ],
  "definition": [
    "GetFiles"
  ]
}

In [ ]:
#Access registered filedataset 

In [ ]:
# ??? Create adls gen2 storage ??
'''# Create Azure Data Lake Storage Gen2 datastore without credentials.
adls2_dstore = Datastore.register_azure_data_lake_gen2(workspace=ws, 
                                                       datastore_name='credentialless_adls2', 
                                                       filesystem='tabular', 
                                                       account_name='myadls2')'''